# Basic LSTM

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:


# Load your data
data = pd.read_csv('data/preprocessed_data.csv', parse_dates=['ds'], index_col='ds')
data = data[['y']]
data

,y
ds,
2014-12-31,270.475000
2015-01-01,275.986987
2015-01-02,276.784833
2015-01-03,279.203802
2015-01-04,279.260000
...,...
2018-12-27,283.708333
2018-12-28,283.936250
2018-12-29,281.977500


In [ ]:
# Hyperparameters
lstm_units = 50
dropout_rate = 0.2
batch_size = 32
epochs = 10
look_back = 60  # Number of previous time steps to use as input variables to predict the next time period


In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['y'].values.reshape(-1, 1))


In [ ]:

# Create the training data
X_train, y_train = [], []
for i in range(look_back, len(scaled_data)):
    X_train.append(scaled_data[i-look_back:i, 0])
    y_train.append(scaled_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape X_train for LSTM input
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(units=lstm_units))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(Dense(1))

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# Making predictions
predicted_temperature = lstm_model.predict(X_train)
predicted_temperature = scaler.inverse_transform(predicted_temperature)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(data.index[look_back:], data['Temperature'][look_back:], color='blue', label='Actual Temperature')
plt.plot(data.index[look_back:], predicted_temperature, color='red', label='Predicted Temperature')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()
